# **IMPORTS**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from skimage.io import imread
import json
import math
from skimage.draw import polygon as pls
import keras.layers as KL
import keras.models as KM
import keras.engine as KE
import keras.backend as K
import tensorflow as tf
import logging
from keras.layers import BatchNormalization as BatchNorm
import skimage.io
import scipy
import skimage.transform
import urllib.request
import shutil
import warnings
from distutils.version import LooseVersion


# **ADDING FILES TO COLAB**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip /content/IDD.zip

# **DATASET CLASS**

In [0]:
class data(object):
  ''' Dataset class to store the class and image info 
 Author - Joseph Arul Raj '''

  def __init__(self):
    self.image_info = []
    self.class_info = [{"source":"Background","id": 0, "name": "BG"}]

  def add_class(self,source,class_id, class_name):
      unwanted=['sky','building','billboard','road']
      for info in self.class_info:
          # if info['name'] == class_name  or unwanted:
          if class_name== info['name'] :
              return False
          if class_name in unwanted:
              return False
      self.class_info.append({"source":source,
                              "id":class_id,
                              "name":class_name,})
      return True

  def add_image(self,image_id, path, **kwargs):
      image_info = {
          "id": image_id,
          "path": path,
      }
      image_info.update(kwargs)
      self.image_info.append(image_info)

# **LOADING THE DATA**

In [0]:

data_dir="/content/IDD"

class anno(data):
'''Class for data preparation.
load_data is to load the annotations from json ffile and to retrive the polygon coordinates for the masks in an image
load_mask is to generate a mask from the obtained coordinates
extract_bboxes is to create a bounding box for each of the generated masks.
Author - Joseph Arul Raj '''
  def load_data(self,data_dir):
    unwanted=['sky','building','billboard','road']
    img_dir=data_dir + "/img/"
    ann_dir=data_dir + "/ann/"
    c_id=1
    pol=[]
    for files in listdir(ann_dir):
      pol1=[]
      ann_path=ann_dir + files
      ann_file=json.load(open(ann_path))
      annotations=list(ann_file.values())[2]
      for i in range(len(annotations)):
        ann=annotations[i]
        name=ann['label']
        added=self.add_class(source=files[:6],class_id=c_id,class_name=name)
        if added:
          c_id +=1
        # return added
      for i in range(len(annotations)):
        cname=annotations[i]['label']
        if cname not in unwanted:
          pol1.append(annotations[i]['polygon'])
      pol.append(pol1)
    pol2=[]
    for i in range(len(pol)):
      pol3=[]
      for j in range(len(pol[i])):
        pol4={'x':[],
               'y':[]}
        for k in range(len(pol[i][j])):
          pol4['x'].append(pol[i][j][k][0])
          pol4['y'].append(pol[i][j][k][1])
        pol3.append(pol4)
      pol2.append(pol3)

    
    for f_name in listdir(img_dir):
      image_id=f_name[:6]
      img_path=img_dir+f_name
      image=imread(img_path)
      ann_path=ann_dir+image_id+'_gtFine_polygons.json'
      height, width = image.shape[:2]
      self.add_image( image_id=image_id, path=img_path, annotation=ann_path,width=width, height=height)

    return pol2

  def load_mask(self,pol):
    masks=[]
    for k in range(len(pol)):
      mask=np.zeros((self.image_info[k]['height'],self.image_info[k]['width'],len(pol[k])))
      for i , p in enumerate(pol[k]):
        rr,cc=pls(p['y'],p['x'])
        mask[rr,cc,i]=1
      masks.append(mask)
    return masks
  def extract_bboxes(self,masks):
    box = np.zeros([masks.shape[-1], 4], dtype=np.int32)
    for j in range(masks.shape[-1]):
        m = masks[:, :, j]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            x2 += 1
            y2 += 1
        else:
            x1, x2, y1, y2 = 0, 0, 0, 0
        box[j] = np.array([y1, x1, y2, x2])
    return box


In [0]:
'''Verify the loaded data '''
train=anno()
da=train.load_data(data_dir)
mask=train.load_mask(da)
bbox=train.extract_bboxes(mask)
img_dir=data_dir + "/img/"
images=np.zeros([2,1080,1920,3])
for filename , i in zip(listdir(img_dir),range(2)):
  im=img_dir+filename
  images[i]=img_to_array(load_img(im,target_size=(1080,1920)))
# plot and see the masks in an image
plt.imshow(images[1]/255)
plt.imshow(mask[1][:, :, 79], alpha=0.5)
plt.imshow(mask[1][:, :, 95], alpha=0.5)
plt.show()

# **ANCHORS GENERATION**

In [0]:
'''Generate anchor on the given feature map
Author - Joseph Arul Raj '''


def norm_boxes_graph(boxes, shape):
    h, w = tf.split(tf.cast(shape, tf.float32), 2)
    scale = tf.concat([h, w, h, w], axis=-1) - tf.constant(1.0)
    shift = tf.constant([0., 0., 1., 1.])
    return tf.divide(boxes - shift, scale)

def compute_backbone_shapes(image_shape,feature_strides):
  return np.array(
        [[int(math.ceil(image_shape[0] / stride)),
            int(math.ceil(image_shape[1] / stride))]
            for stride in feature_strides])
def generate_anchors(scales, ratios, shape, feature_stride, anchor_stride):
    # Get all combinations of scales and ratios
    scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
    scales = scales.flatten()
    ratios = ratios.flatten()

    # Enumerate heights and widths from scales and ratios
    heights = scales / np.sqrt(ratios)
    widths = scales * np.sqrt(ratios)

    # Enumerate shifts in feature space
    shifts_y = np.arange(0, shape[0], anchor_stride) * feature_stride
    shifts_x = np.arange(0, shape[1], anchor_stride) * feature_stride
    shifts_x, shifts_y = np.meshgrid(shifts_x, shifts_y)

    # Enumerate combinations of shifts, widths, and heights
    box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
    box_heights, box_centers_y = np.meshgrid(heights, shifts_y)

    # Reshape to get a list of (y, x) and a list of (h, w)
    box_centers = np.stack(
        [box_centers_y, box_centers_x], axis=2).reshape([-1, 2])
    box_sizes = np.stack([box_heights, box_widths], axis=2).reshape([-1, 2])

    # Convert to corner coordinates (y1, x1, y2, x2)
    boxes = np.concatenate([box_centers - 0.5 * box_sizes,
                            box_centers + 0.5 * box_sizes], axis=1)
    return boxes


def generate_pyramid_anchors(scales, ratios, feature_shapes, feature_strides,
                             anchor_stride):
   # Anchors
    # [anchor_count, (y1, x1, y2, x2)]
    anchors = []
    for i in range(len(scales)):
        anchors.append(generate_anchors(scales[i], ratios, feature_shapes[i],
                                        feature_strides[i], anchor_stride))
    return np.concatenate(anchors, axis=0)

# **BACKBONE CNN -RESNET**

In [0]:
'''Resnet graph for the backbone cnn
author -Asad Tanveer'''

# The identity_block is the block that has no conv layer at shortcut
def identity_block(input_tensor, kernel_size, filters,train_bn=True):
   nb_filter1, nb_filter2, nb_filter3 = filters
   x = KL.Conv2D(nb_filter1, (1, 1),use_bias=True)(input_tensor)
   x = KL.BatchNormalization()(x,training=True)
   x = KL.Activation('relu')(x)

   x = KL.Conv2D(nb_filter2, (kernel_size, kernel_size), padding='same',use_bias=True)(x)
   x = KL.BatchNormalization()(x,training=True)
   x = KL.Activation('relu')(x)

   x = KL.Conv2D(nb_filter3, (1, 1),use_bias=True)(x)
   x = KL.BatchNormalization()(x,training=True)

   x = KL.Add()([x, input_tensor])
   x = KL.Activation('relu')(x)
   return x
# conv_block is the block that has a conv layer at shortcut
def conv_block(input_tensor, kernel_size, filters,strides=(2, 2), use_bias=True, train_bn=True):
    nb_filter1, nb_filter2, nb_filter3 = filters
    x = KL.Conv2D(nb_filter1, (1, 1), strides=strides, use_bias=use_bias)(input_tensor)
    x = KL.BatchNormalization()(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.Conv2D(nb_filter2, (kernel_size, kernel_size), padding='same', use_bias=use_bias)(x)
    x = KL.BatchNormalization()(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.Conv2D(nb_filter3, (1, 1), use_bias=use_bias)(x)
    x = KL.BatchNormalization()(x, training=train_bn)

    shortcut = KL.Conv2D(nb_filter3, (1, 1), strides=strides, use_bias=use_bias)(input_tensor)
    shortcut = KL.BatchNormalization()(shortcut, training=train_bn)

    x = KL.Add()([x, shortcut])
    x = KL.Activation('relu')(x)
    return x
# Build a ResNet graph.
def resnet_graph(input_image, train_bn=True):
    # Stage 1
    x = KL.ZeroPadding2D((3, 3))(input_image)
    x = KL.Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=True)(x)
    x = KL.BatchNormalization()(x, training=train_bn)
    x = KL.Activation('relu')(x)
    C1 = x = KL.MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)
    # Stage 2
    x = conv_block(x, 3, [64, 64, 256],strides=(1, 1), train_bn=train_bn)
    x = identity_block(x, 3, [64, 64, 256],train_bn=train_bn)
    C2 = x = identity_block(x, 3, [64, 64, 256], train_bn=train_bn)
    # Stage 3
    x = conv_block(x, 3, [128, 128, 512], train_bn=train_bn)
    x = identity_block(x, 3, [128, 128, 512], train_bn=train_bn)
    x = identity_block(x, 3, [128, 128, 512], train_bn=train_bn)
    C3 = x = identity_block(x, 3, [128, 128, 512], train_bn=train_bn)
    # Stage 4
    x = conv_block(x, 3, [256, 256, 1024], train_bn=train_bn)
    for i in range(5):
        x = identity_block(x, 3, [256, 256, 1024], train_bn=train_bn)
    C4 = x
    # Stage 5
    x = conv_block(x, 3, [512, 512, 2048], train_bn=train_bn)
    x = identity_block(x, 3, [512, 512, 2048], train_bn=train_bn)
    C5 = x = identity_block(x, 3, [512, 512, 2048], train_bn=train_bn)
    return [C1, C2, C3, C4, C5]

## **CONSTRUCT A FEATURE PYRAMID NETWORK**

In [0]:
'''The FPN that connects the backbone cnn with RPN
author - Joseph Arul Raj '''

# Build dummy imputs
img_shape=[1024,1024,3]
input_image=KL.Input(shape=[None,None,3])
input_rpn_bbox = KL.Input(shape=[None, 4], name="input_rpn_bbox", dtype=tf.float32)
input_gt_boxes = KL.Input(shape=[None, 4], name="input_gt_boxes", dtype=tf.float32)
gt_boxes = KL.Lambda(lambda x: norm_boxes_graph(x, K.shape(input_image)[1:3]))(input_gt_boxes)
input_gt_masks = KL.Input(shape=[img_shape[0], img_shape[1], 100],name="input_gt_masks", dtype=bool)

# Get the feature maps from Resnet graph
_, C2, C3, C4, C5 = resnet_graph(input_image)

# FPN graph

P5 = KL.Conv2D(256, (1, 1), name='fpn_c5p5')(C5)
P4 = KL.Add(name="fpn_p4add")([
    KL.UpSampling2D(size=(2, 2), name="fpn_p5upsampled")(P5),
    KL.Conv2D(256, (1, 1), name='fpn_c4p4')(C4)])
P3 = KL.Add(name="fpn_p3add")([
    KL.UpSampling2D(size=(2, 2), name="fpn_p4upsampled")(P4),
    KL.Conv2D(256, (1, 1), name='fpn_c3p3')(C3)])
P2 = KL.Add(name="fpn_p2add")([
    KL.UpSampling2D(size=(2, 2), name="fpn_p3upsampled")(P3),
    KL.Conv2D(256, (1, 1), name='fpn_c2p2')(C2)])
# Attach 3x3 conv to all P layers to get the final feature maps.
P2 = KL.Conv2D(256, (3, 3), padding="SAME", name="fpn_p2")(P2)
P3 = KL.Conv2D(256, (3, 3), padding="SAME", name="fpn_p3")(P3)
P4 = KL.Conv2D(256, (3, 3), padding="SAME", name="fpn_p4")(P4)
P5 = KL.Conv2D(256, (3, 3), padding="SAME", name="fpn_p5")(P5)
# P6 is used for the 5th anchor scale in RPN. Generated by
# subsampling from P5 with stride of 2.
P6 = KL.MaxPooling2D(pool_size=(1, 1), strides=2, name="fpn_p6")(P5)

rpn_feature_maps = [P2, P3, P4, P5, P6]
mrcnn_feature_maps = [P2, P3, P4, P5]

# **RPN**

In [0]:
'''Region propasal graph - generates two outputs. One containing the object score and the other containing the bounding box coordinates
author - Asad Tanveer'''

def rpn_graph(feature_map, anchors_per_location, anchor_stride):
    shared = KL.Conv2D(512, (3, 3), padding='same', activation='relu',
                       strides=anchor_stride,
                       name='rpn_conv_shared')(feature_map)

    # Anchor Score. [batch, height, width, anchors per location * 2].
    x = KL.Conv2D(2 * anchors_per_location, (1, 1), padding='valid',
                  activation='linear', name='rpn_class_raw')(shared)

    # Reshape to [batch, anchors, 2]
    rpn_class_logits = KL.Lambda(
        lambda t: tf.reshape(t, [tf.shape(t)[0], -1, 2]))(x)

    # Softmax on last dimension of BG/FG.
    rpn_probs = KL.Activation(
        "softmax", name="rpn_class_xxx")(rpn_class_logits)

    # Bounding box refinement. [batch, H, W, anchors per location * depth]
    # where depth is [x, y, log(w), log(h)]
    x = KL.Conv2D(anchors_per_location * 4, (1, 1), padding="valid",
                  activation='linear', name='rpn_bbox_pred')(shared)

    # Reshape to [batch, anchors, 4]
    rpn_bbox = KL.Lambda(lambda t: tf.reshape(t, [tf.shape(t)[0], -1, 4]))(x)

    return [rpn_class_logits, rpn_probs, rpn_bbox]


input_feature_map = KL.Input(shape=[None, None, 256])
outputs = rpn_graph(input_feature_map, 3, 1)
rpn= KM.Model([input_feature_map], outputs, name="rpn_model")
# Map the feature maps generated from the FPN with the RPN graph
layer_outputs = []  # list of lists
for p in rpn_feature_maps:
    layer_outputs.append(rpn([p]))
output_names = ["rpn_class_logits", "rpn_class", "rpn_bbox"]
outputs = list(zip(*layer_outputs))
outputs = [KL.Concatenate(axis=1, name=n)(list(o))
            for o, n in zip(outputs, output_names)]

rpn_class_logits, rpn_class, rpn_bbox = outputs

# **PROPOSAL LAYER**

In [0]:
'''Proposal layer is to select 2000 boxes from the generated 200k boxes
Author - Joseph Arul Raj '''


def apply_box_deltas_graph(boxes, deltas):
    """Applies the given deltas to the given boxes.
    boxes: [N, (y1, x1, y2, x2)] boxes to update
    deltas: [N, (dy, dx, log(dh), log(dw))] refinements to apply
    """
    # Convert to y, x, h, w
    height = boxes[:, 2] - boxes[:, 0]
    width = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + 0.5 * height
    center_x = boxes[:, 1] + 0.5 * width
    # Apply deltas
    center_y += deltas[:, 0] * height
    center_x += deltas[:, 1] * width
    height *= tf.exp(deltas[:, 2])
    width *= tf.exp(deltas[:, 3])
    # Convert back to y1, x1, y2, x2
    y1 = center_y - 0.5 * height
    x1 = center_x - 0.5 * width
    y2 = y1 + height
    x2 = x1 + width
    result = tf.stack([y1, x1, y2, x2], axis=1, name="apply_box_deltas_out")
    return result

# clip the boxes to the image height and width
def clip_boxes_graph(boxes, window):
    """
    boxes: [N, (y1, x1, y2, x2)]
    window: [4] in the form y1, x1, y2, x2
    """
    # Split
    wy1, wx1, wy2, wx2 = tf.split(window, 4)
    y1, x1, y2, x2 = tf.split(boxes, 4, axis=1)
    # Clip
    y1 = tf.maximum(tf.minimum(y1, wy2), wy1)
    x1 = tf.maximum(tf.minimum(x1, wx2), wx1)
    y2 = tf.maximum(tf.minimum(y2, wy2), wy1)
    x2 = tf.maximum(tf.minimum(x2, wx2), wx1)
    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    clipped.set_shape((clipped.shape[0], 4))
    return clipped

class ProposalLayer(KE.Layer):
    def __init__(self, proposal_count, nms_threshold, **kwargs):
        super(ProposalLayer, self).__init__(**kwargs)
        self.proposal_count = proposal_count
        self.nms_threshold = nms_threshold

    def call(self, inputs):
        # Box Scores. Use the foreground class confidence. [Batch, num_rois, 1]
        sc=inputs[0][:,:,1]
        deltas=inputs[1]
        anc =inputs[2][1]
        RPN_BBOX_STD_DEV = np.array([0.1, 0.1, 0.2, 0.2])
        deltas = deltas * np.reshape(RPN_BBOX_STD_DEV, [1, 1, 4])
        proposals=[]
        for i in range(2):
          cls_score=sc[i,:]
          # select indices for top 6000 boxes based on the object score
          ix = tf.math.top_k(cls_score, 6000, sorted=True,name="top_anchors").indices
          # select the scores,boxes and anchors from the 6000 indices
          bbo=deltas[i,:,:]
          sco=tf.gather(cls_score,ix)
          bbb=tf.gather(bbo,ix)
          aaa=tf.gather(anc,ix)
          anbo=apply_box_deltas_graph(aaa,bbb)
          window = np.array([0, 0, 1, 1], dtype=np.float32)
          cl_bo=clip_boxes_graph(anbo,window)
          indices = tf.image.non_max_suppression(cl_bo,sco, self.proposal_count,self.nms_threshold)
          proposal = tf.gather(cl_bo, indices)
          proposals.append(proposal)
        rpn_rois=tf.convert_to_tensor(proposals)
        return rpn_rois

    def compute_output_shape(self, input_shape):
        return (None, self.proposal_count, 4)


# **TARGET DETECTION**

In [0]:
# To slice the given inputs and pass them as batches

def batch_slice(inputs, graph_fn, batch_size, names=None):
    if not isinstance(inputs, list):
        inputs = [inputs]

    outputs = []
    for i in range(batch_size):
        inputs_slice = [x[i] for x in inputs]
        output_slice = graph_fn(*inputs_slice)
        if not isinstance(output_slice, (tuple, list)):
            output_slice = [output_slice]
        outputs.append(output_slice)
    # Change outputs from a list of slices where each is
    # a list of outputs to a list of outputs and each has
    # a list of slices
    outputs = list(zip(*outputs))

    if names is None:
        names = [None] * len(outputs)

    result = [tf.stack(o, axis=0, name=n)
              for o, n in zip(outputs, names)]
    if len(result) == 1:
        result = result[0]

    return result


In [0]:
'''Tatget detection layer is to choose 200(both positive and negetive) boxes for training
Author - Asad Tanveer'''

def box_refinement_graph(box, gt_box):
    """Compute refinement needed to transform box to gt_box.
    box and gt_box are [N, (y1, x1, y2, x2)]
    """
    box = tf.cast(box, tf.float32)
    gt_box = tf.cast(gt_box, tf.float32)

    height = box[:, 2] - box[:, 0]
    width = box[:, 3] - box[:, 1]
    center_y = box[:, 0] + 0.5 * height
    center_x = box[:, 1] + 0.5 * width

    gt_height = gt_box[:, 2] - gt_box[:, 0]
    gt_width = gt_box[:, 3] - gt_box[:, 1]
    gt_center_y = gt_box[:, 0] + 0.5 * gt_height
    gt_center_x = gt_box[:, 1] + 0.5 * gt_width

    dy = (gt_center_y - center_y) / height
    dx = (gt_center_x - center_x) / width
    dh = tf.log(gt_height / height)
    dw = tf.log(gt_width / width)

    result = tf.stack([dy, dx, dh, dw], axis=1)
    return result

def overlaps_graph(boxes1, boxes2):
    """Computes IoU overlaps between two sets of boxes.
    boxes1, boxes2: [N, (y1, x1, y2, x2)].
    """
    # 1. Tile boxes2 and repeat boxes1. This allows us to compare
    # every boxes1 against every boxes2 without loops.
    # TF doesn't have an equivalent to np.repeat() so simulate it
    # using tf.tile() and tf.reshape.
    b1 = tf.reshape(tf.tile(tf.expand_dims(boxes1, 1),
                            [1, 1, tf.shape(boxes2)[0]]), [-1, 4])
    b2 = tf.tile(boxes2, [tf.shape(boxes1)[0], 1])
    # 2. Compute intersections
    b1_y1, b1_x1, b1_y2, b1_x2 = tf.split(b1, 4, axis=1)
    b2_y1, b2_x1, b2_y2, b2_x2 = tf.split(b2, 4, axis=1)
    y1 = tf.maximum(b1_y1, b2_y1)
    x1 = tf.maximum(b1_x1, b2_x1)
    y2 = tf.minimum(b1_y2, b2_y2)
    x2 = tf.minimum(b1_x2, b2_x2)
    intersection = tf.maximum(x2 - x1, 0) * tf.maximum(y2 - y1, 0)
    # 3. Compute unions
    b1_area = (b1_y2 - b1_y1) * (b1_x2 - b1_x1)
    b2_area = (b2_y2 - b2_y1) * (b2_x2 - b2_x1)
    union = b1_area + b2_area - intersection
    # 4. Compute IoU and reshape to [boxes1, boxes2]
    iou = intersection / union
    overlaps = tf.reshape(iou, [tf.shape(boxes1)[0], tf.shape(boxes2)[0]])
    return overlaps


def detection_targets_graph(proposals, gt_boxes, gt_masks):

    # Compute overlaps matrix [proposals, gt_boxes]
    overlaps = overlaps_graph(proposals, gt_boxes)

    # Compute overlaps with crowd boxes [proposals, crowd_boxes]
    # crowd_overlaps = overlaps_graph(proposals, crowd_boxes)
    # crowd_iou_max = tf.reduce_max(crowd_overlaps, axis=1)
    # no_crowd_bool = (crowd_iou_max < 0.001)

    # Determine positive and negative ROIs
    roi_iou_max = tf.reduce_max(overlaps, axis=1)
    # 1. Positive ROIs are those with >= 0.5 IoU with a GT box
    positive_roi_bool = (roi_iou_max >= 0.5)
    positive_indices = tf.where(positive_roi_bool)[:, 0]
    # 2. Negative ROIs are those with < 0.5 with every GT box. Skip crowds.
    negative_indices = tf.where(roi_iou_max < 0.5)[:, 0]

    # Subsample ROIs. Aim for 33% positive
    # Positive ROIs
    positive_count = int(200 *0.33)
    positive_indices = tf.random_shuffle(positive_indices)[:positive_count]
    positive_count = tf.shape(positive_indices)[0]
    # Negative ROIs. Add enough to maintain positive:negative ratio.
    r = 1.0 / 0.33
    negative_count = tf.cast(r * tf.cast(positive_count, tf.float32), tf.int32) - positive_count
    negative_indices = tf.random_shuffle(negative_indices)[:negative_count]
    # Gather selected ROIs
    positive_rois = tf.gather(proposals, positive_indices)
    negative_rois = tf.gather(proposals, negative_indices)

    # Assign positive ROIs to GT boxes.
    positive_overlaps = tf.gather(overlaps, positive_indices)
    roi_gt_box_assignment = tf.cond(
        tf.greater(tf.shape(positive_overlaps)[1], 0),
        true_fn = lambda: tf.argmax(positive_overlaps, axis=1),
        false_fn = lambda: tf.cast(tf.constant([]),tf.int64)
    )
    roi_gt_boxes = tf.gather(gt_boxes, roi_gt_box_assignment)
    # roi_gt_class_ids = tf.gather(gt_class_ids, roi_gt_box_assignment)

    # Compute bbox refinement for positive ROIs
    BBOX_STD_DEV = np.array([0.1, 0.1, 0.2, 0.2])
    deltas = box_refinement_graph(positive_rois, roi_gt_boxes)
    deltas /= BBOX_STD_DEV

    # Assign positive ROIs to GT masks
    # Permute masks to [N, height, width, 1]
    transposed_masks = tf.expand_dims(tf.transpose(gt_masks, [2, 0, 1]), -1)
    # Pick the right mask for each ROI
    roi_masks = tf.gather(transposed_masks, roi_gt_box_assignment)
    MASK_SHAPE=[28,28]
    # Compute mask targets
    boxes = positive_rois
    box_ids = tf.range(0, tf.shape(roi_masks)[0])
    masks = tf.image.crop_and_resize(tf.cast(roi_masks, tf.float32), boxes,
                                     box_ids,
                                     MASK_SHAPE)
    # Remove the extra dimension from masks.
    masks = tf.squeeze(masks, axis=3)

    # Threshold mask pixels at 0.5 to have GT masks be 0 or 1 to use with
    # binary cross entropy loss.
    masks = tf.round(masks)

    # Append negative ROIs and pad bbox deltas and masks that
    # are not used for negative ROIs with zeros.
    rois = tf.concat([positive_rois, negative_rois], axis=0)
    N = tf.shape(negative_rois)[0]
    P = tf.maximum(200 - tf.shape(rois)[0], 0)
    rois = tf.pad(rois, [(0, P), (0, 0)])
    roi_gt_boxes = tf.pad(roi_gt_boxes, [(0, N + P), (0, 0)])
    # roi_gt_class_ids = tf.pad(roi_gt_class_ids, [(0, N + P)])
    deltas = tf.pad(deltas, [(0, N + P), (0, 0)])
    masks = tf.pad(masks, [[0, N + P], (0, 0), (0, 0)])

    return rois,  deltas, masks


class DetectionTargetLayer(KL.Layer):
    
    def __init__(self, **kwargs):
        super(DetectionTargetLayer, self).__init__(**kwargs)
        # self.config = config

    def call(self, inputs):
        proposals = inputs[0]
        # gt_class_ids = inputs[1]
        gt_boxes = inputs[1]
        gt_masks = inputs[2]

        # Slice the batch and run a graph for each slice
        # TODO: Rename target_bbox to target_deltas for clarity
        names = ["rois", "target_class_ids", "target_bbox", "target_mask"]
        outputs = batch_slice([proposals, gt_boxes, gt_masks],lambda  x, y, z: detection_targets_graph( x, y, z),2, names=names)
        return outputs

    def compute_output_shape(self, input_shape):
        return [
            (None, 200, 4),  # rois
            # (None, 200),  # class_ids
            (None, 200, 4),  # deltas
            (None, 200,28,28)  # masks
        ]
    def compute_mask(self, inputs, mask=None):
        return [None, None, None, None]


# **GENERATE ROIS**

In [0]:
# Call the proposal layer to select 2000 boxes

rpn_rois = ProposalLayer(proposal_count=2000,nms_threshold=0.7,name="ROI")([rpn_class, rpn_bbox, anchors])

# call the detection target layer to choose 200 samples from 2000 boxes

rois, target_bbox, target_mask =DetectionTargetLayer(name="proposal_targets")([rpn_rois, gt_boxes, input_gt_masks])


**ROI ALLIGN**

In [0]:
'''Applies ROI Allign to the feature maps generated by the FPN 
Author - Joseph Arul Raj '''

def log2_graph(x):
    """Implementation of Log2. TF doesn't have a native implementation."""
    return tf.log(x) / tf.log(2.0)


class ROIAlign(KL.Layer):
    def __init__(self, pool_shape, **kwargs):
        super(ROIAlign, self).__init__(**kwargs)
        self.pool_shape = tuple(pool_shape)

    def call(self, inputs):
        # Crop boxes [batch, num_boxes, (y1, x1, y2, x2)] in normalized coords
        boxes = inputs[0]

        # Image meta
        # Holds details about the image. See compose_image_meta()
        # image_meta = inputs[1]

        # Feature Maps. List of feature maps from different level of the
        # feature pyramid. Each is [batch, height, width, channels]
        feature_maps = inputs[1:]

        # Assign each ROI to a level in the pyramid based on the ROI area.
        y1, x1, y2, x2 = tf.split(boxes, 4, axis=2)
        h = y2 - y1
        w = x2 - x1
        # Use shape of first image. Images in a batch must have the same size.
        image_shape = [1024,1024,3]
        # Equation 1 in the Feature Pyramid Networks paper. Account for
        # the fact that our coordinates are normalized here.
        # e.g. a 224x224 ROI (in pixels) maps to P4
        image_area = tf.cast(image_shape[0] * image_shape[1], tf.float32)
        roi_level = log2_graph(tf.sqrt(h * w) / (224.0 / tf.sqrt(image_area)))
        roi_level = tf.minimum(5, tf.maximum(
            2, 4 + tf.cast(tf.round(roi_level), tf.int32)))
        roi_level = tf.squeeze(roi_level, 2)

        # Loop through levels and apply ROI poinput_gt_masksoling to each. P2 to P5.
        pooled = []
        box_to_level = []
        for i, level in enumerate(range(2, 6)):
            ix = tf.where(tf.equal(roi_level, level))
            level_boxes = tf.gather_nd(boxes, ix)

            # Box indices for crop_and_resize.
            box_indices = tf.cast(ix[:, 0], tf.int32)

            # Keep track of which box is mapped to which level
            box_to_level.append(ix)

            # Stop gradient propogation to ROI proposals
            level_boxes = tf.stop_gradient(level_boxes)
            box_indices = tf.stop_gradient(box_indices)

            pooled.append(tf.image.crop_and_resize(
                feature_maps[i], level_boxes, box_indices, self.pool_shape,
                method="bilinear"))

        # Pack pooled features into one tensor
        pooled = tf.concat(pooled, axis=0)

        # Pack box_to_level mapping into one array and add another
        # column representing the order of pooled boxes
        box_to_level = tf.concat(box_to_level, axis=0)
        box_range = tf.expand_dims(tf.range(tf.shape(box_to_level)[0]), 1)
        box_to_level = tf.concat([tf.cast(box_to_level, tf.int32), box_range],
                                 axis=1)

        # Rearrange pooled features to match the order of the original boxes
        # Sort box_to_level by batch then box index
        # TF doesn't have a way to sort by two columns, so merge them and sort.
        sorting_tensor = box_to_level[:, 0] * 100000 + box_to_level[:, 1]
        ix = tf.nn.top_k(sorting_tensor, k=tf.shape(
            box_to_level)[0]).indices[::-1]
        ix = tf.gather(box_to_level[:, 2], ix)
        pooled = tf.gather(pooled, ix)

        # Re-add the batch dimension
        shape = tf.concat([tf.shape(boxes)[:2], tf.shape(pooled)[1:]], axis=0)
        pooled = tf.reshape(pooled, shape)
        return pooled

    def compute_output_shape(self, input_shape):
        return input_shape[0][:2] + self.pool_shape + (input_shape[2][-1], )

# **MRCNN CLASSIFIER AND MASK HEADS**

In [0]:

'''Classifier graph to finally classify the object classes and to find bbox regression for them
Author - Asad Tanveer '''
def fpn_classifier_graph(rois, feature_maps,pool_size, num_classes, train_bn=True,fc_layers_size=1024):
    # ROI Pooling
    # Shape: [batch, num_rois, POOL_SIZE, POOL_SIZE, channels]
    # x = roi_allign([rois]+feature_maps)
    x = ROIAlign([pool_size,pool_size])([rois] + mrcnn_feature_maps)
    # x = PyramidROIAlign([pool_size, pool_size],
    #                     name="roi_align_classifier")([rois, image] + feature_maps)
    # Two 1024 FC layers (implemented with Conv2D for consistency)
    x = KL.TimeDistributed(KL.Conv2D(fc_layers_size, (pool_size, pool_size), padding="valid"),
                           name="mrcnn_class_conv1")(x)
    x = KL.TimeDistributed(BatchNorm(), name='mrcnn_class_bn1')(x, training=train_bn)
    x = KL.Activation('relu')(x)
    x = KL.TimeDistributed(KL.Conv2D(fc_layers_size, (1, 1)),
                           name="mrcnn_class_conv2")(x)
    x = KL.TimeDistributed(BatchNorm(), name='mrcnn_class_bn2')(x, training=train_bn)
    x = KL.Activation('relu')(x)

    shared = KL.Lambda(lambda x: K.squeeze(K.squeeze(x, 3), 2),
                       name="pool_squeeze")(x)

    # Classifier head
    mrcnn_class_logits = KL.TimeDistributed(KL.Dense(num_classes),
                                            name='mrcnn_class_logits')(shared)
    mrcnn_probs = KL.TimeDistributed(KL.Activation("softmax"),
                                     name="mrcnn_class")(mrcnn_class_logits)

    # BBox head
    # [batch, num_rois, NUM_CLASSES * (dy, dx, log(dh), log(dw))]
    x = KL.TimeDistributed(KL.Dense(num_classes * 4, activation='linear'),
                           name='mrcnn_bbox_fc')(shared)
    # Reshape to [batch, num_rois, NUM_CLASSES, (dy, dx, log(dh), log(dw))]
    s = K.int_shape(x)
    # mrcnn_bbox=x
    mrcnn_bbox = KL.Reshape((s[1], num_classes, 4), name="mrcnn_bbox")(x)
    # mrcnn_bbox=x
    return mrcnn_class_logits, mrcnn_probs, mrcnn_bbox

'''Mask head to generate a mask for the identified class with the bbox
Author - Joseph Arul Raj '''
def build_fpn_mask_graph(rois, feature_maps,
                         pool_size, num_classes, train_bn=True):
    # ROI Pooling
    # Shape: [batch, num_rois, MASK_POOL_SIZE, MASK_POOL_SIZE, channels]
    # x = ROIAlign([pool_size,rois,feature_maps])
    x = ROIAlign([pool_size,pool_size])([rois] + mrcnn_feature_maps)
    # PyramidROIAlign([pool_size, pool_size],
    #                     name="roi_align_mask")([rois, image] + feature_maps)

    # Conv layers
    x = KL.TimeDistributed(KL.Conv2D(256, (3, 3), padding="same"),
                           name="mrcnn_mask_conv1")(x)
    x = KL.TimeDistributed(BatchNorm(),
                           name='mrcnn_mask_bn1')(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.TimeDistributed(KL.Conv2D(256, (3, 3), padding="same"),
                           name="mrcnn_mask_conv2")(x)
    x = KL.TimeDistributed(BatchNorm(),
                           name='mrcnn_mask_bn2')(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.TimeDistributed(KL.Conv2D(256, (3, 3), padding="same"),
                           name="mrcnn_mask_conv3")(x)
    x = KL.TimeDistributed(BatchNorm(),
                           name='mrcnn_mask_bn3')(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.TimeDistributed(KL.Conv2D(256, (3, 3), padding="same"),
                           name="mrcnn_mask_conv4")(x)
    x = KL.TimeDistributed(BatchNorm(),
                           name='mrcnn_mask_bn4')(x, training=train_bn)
    x = KL.Activation('relu')(x)

    x = KL.TimeDistributed(KL.Conv2DTranspose(256, (2, 2), strides=2, activation="relu"),
                           name="mrcnn_mask_deconv")(x)
    x = KL.TimeDistributed(KL.Conv2D(num_classes, (1, 1), strides=1, activation="sigmoid"),
                           name="mrcnn_mask")(x)
    return x

In [0]:
# Classifier head

mrcnn_class_logits, mrcnn_class, mrcnn_bbox= fpn_classifier_graph(rois, mrcnn_feature_maps,7, 2)

# Mask head

mrcnn_mask = build_fpn_mask_graph(rois, mrcnn_feature_maps,14,2)

## **GENERATE DATA FOR TRAINING**

In [0]:
'''Since our model has to scale down the image to six times during training,we need the inputs which can be divided by 2 six times .
so we need to resize the inputs 
Author - Asad Tanveer'''

def resize(image, output_shape, order=1, mode='constant', cval=0, clip=True,
           preserve_range=False, anti_aliasing=False, anti_aliasing_sigma=None):

    if LooseVersion(skimage.__version__) >= LooseVersion("0.14"):
        # New in 0.14: anti_aliasing. Default it to False for backward
        # compatibility with skimage 0.13.
        return skimage.transform.resize(
            image, output_shape,
            order=order, mode=mode, cval=cval, clip=clip,
            preserve_range=preserve_range, anti_aliasing=anti_aliasing,
            anti_aliasing_sigma=anti_aliasing_sigma)
    else:
        return skimage.transform.resize(
            image, output_shape,
            order=order, mode=mode, cval=cval, clip=clip,
            preserve_range=preserve_range)


def resize_image(image, min_dim=800, max_dim=1024, min_scale=None, mode="square"):
    # Keep track of image dtype and return results in the same dtype
    image_dtype = image.dtype

    # Scale?
    if min_dim:
        # Scale up but not down
        scale = max(1, min_dim / min(h, w))
    if min_scale and scale < min_scale:
        scale = min_scale

    # Does it exceed max dim?
    if max_dim and mode == "square":
        image_max = max(h, w)
        if round(image_max * scale) > max_dim:
            scale = max_dim / image_max

    # Resize image using bilinear interpolation
    if scale != 1:
        image = resize(image, (round(h * scale), round(w * scale)),
                       preserve_range=True)

    # Need padding or cropping?
    if mode == "square":
        # Get new height and width
        h, w = image.shape[:2]
        top_pad = (max_dim - h) // 2
        bottom_pad = max_dim - h - top_pad
        left_pad = (max_dim - w) // 2
        right_pad = max_dim - w - left_pad
        padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
        image = np.pad(image, padding, mode='constant', constant_values=0)
        window = (top_pad, left_pad, h + top_pad, w + left_pad)


    else:
        raise Exception("Mode {} not supported".format(mode))
    return image.astype(image_dtype), window, scale, padding


def resize_mask(mask, scale, padding):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mask = scipy.ndimage.zoom(mask, zoom=[scale, scale, 1], order=0)
    mask = np.pad(mask, padding, mode='constant', constant_values=0)
    return mask


In [0]:
'''A python generator function to generate the training data
author - Joseph Arul Raj '''

def mold_image(images):
    """normalize the pixel values
    """
    MEAN_PIXEL = np.array([123.7, 116.8, 103.9])
    return images.astype(np.float32) - MEAN_PIXEL
def data_generator(batch_size=2):
    
    b = 0  # batch item index
    image_index = -1
    # image_ids = np.copy(dataset.image_ids)
    error_count = 0

    # Anchors
    # [anchor_count, (y1, x1, y2, x2)]
    MAX_GT_INSTANCES =100
    img_shape=[1024,1024,3]
    feature_strides=[4, 8, 16, 32, 64]
    backbone_shapes = compute_backbone_shapes(img_shape,feature_strides)
    anchors=generate_pyramid_anchors((32, 64, 128, 256, 512),[0.5, 1, 2],backbone_shapes,feature_strides,1)
    train=anno()
    da=train.load_data(data_dir)
    print('............')
    masks=train.load_mask(da)
    print('............')
    # bboxes=train.extract_bboxes(masks)
    print('............')
    img_dir=data_dir + "/img/"
    images=np.zeros([2,1080,1920,3])
    for filename , i in zip(listdir(img_dir),range(2)):
      im=img_dir+filename
      print(i,filename)
      images[i]=img_to_array(load_img(im,target_size=(1080,1920)))

    info=train.image_info

    # Keras requires a generator to run indefinitely.
    while True:
        print('Loop started....................................................................................',image_index)
        try:
            # Increment index to pick next image. Shuffle if at the start of an epoch.
            print('started....')
            image_index = (image_index + 1)

            image=images[image_index]
            image, window, scale, padding= resize_image(image,min_dim=800,min_scale=0,max_dim=1024,mode='square')
            print('image',image.shape)
            gt_masks=masks[image_index]
            gt_masks= resize_mask(gt_masks, scale, padding, crop)
            print('masks',gt_masks.shape)
            gt_boxes=train.extract_bboxes(gt_masks)
            print('bbox',gt_boxes.shape)

            # Init batch arrays
            if b == 0:
                print('batching...')
                batch_images = np.zeros(
                    (batch_size,) + image.shape, dtype=np.float32)
                print('im')
                # batch_gt_class_ids = np.zeros(
                #     (batch_size, MAX_GT_INSTANCES), dtype=np.int32)
                batch_gt_boxes = np.zeros(
                    (batch_size,MAX_GT_INSTANCES, 4), dtype=np.int32)
                print('bb')
                batch_gt_masks = np.zeros(
                    (batch_size, gt_masks.shape[0], gt_masks.shape[1],
                     MAX_GT_INSTANCES), dtype=gt_masks.dtype)
    

                print('batch created....')
           # If more instances 1080than fits in the array, sub-sample from them.
            print('gt instances present : ',gt_boxes.shape[0])
            if gt_boxes.shape[0] >MAX_GT_INSTANCES:
                ids = np.random.choice(
                    np.arange(gt_boxes.shape[0]), MAX_GT_INSTANCES, replace=False)
                print('ids',ids.shape)
                print('...........',ids[1],ids[2])
                # gt_class_ids = gt_class_ids[ids]
                gt_boxes = gt_boxes[ids]
                print('...........',ids[3],ids[5],gt_masks.shape)
                gt_masks = gt_masks[:, :, ids]
                print('instances<=100')
            

            # Add to batch

            batch_images[b] = mold_image(image.astype(np.float32))
            print('batch image')
            # batch_gt_class_ids[b, :gt_class_ids.shape[0]] = gt_class_ids
            batch_gt_boxes[b, :gt_boxes.shape[0]] = gt_boxes
            print('batch bbox')
            batch_gt_masks[b, :, :, :gt_masks.shape[-1]] = gt_masks
            print('batch mask')


            b += 1

            # Batch full?
            if b >= batch_size:
                print('Batch full')
                inputs = [batch_images, batch_gt_boxes, batch_gt_masks]

                yield inputs

                # start a new batch
                b = 0
        except (GeneratorExit, KeyboardInterrupt):
            raise
        except:
            # Log it and skip the image
            logging.exception("Error processing image {}".format(info[image_index]))
            error_count += 1
            if error_count > 5:
                raise


# **MODEL BUILDING AND TRAINING**

MODEL **BUILDING**

In [0]:
output_rois = KL.Lambda(lambda x: x * 1, name="output_rois")(rois)

inputs = [input_image, input_gt_boxes, input_gt_masks]
outputs = [rpn_class_logits, rpn_class, rpn_bbox, mrcnn_class_logits, mrcnn_class, mrcnn_bbox,mrcnn_mask, rpn_rois, output_rois]
model = KM.Model(inputs, outputs, name='mask_rcnn')
model.summary()

**COMPILE** **IT**

In [0]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy')

**TRAINING**

In [0]:
# Get the training data

train_datas=data_generator()

# Fit the data

model.fit(train_datas,steps_per_epoch=1)